In [36]:
# importing relevant libraries:

In [37]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData

In [38]:
# connecting to postgres db with sqlalchemy.create_engine:

In [39]:
engine = create_engine('postgresql://:@:/')

In [40]:
# reading .csv file to pandas.DataFrame, 
# performing initial data cleansing, 
# and loading data to db with DataFrame.to_sql:  

In [ ]:
with open('fifa_data_2.csv', 'r', encoding="utf8") as file:
    df = pd.read_csv(file, index_col='ID')
df.columns = df.columns.str.lower()
df.rename(columns={"club logo":"club_logo", "value":"valueM€"}, inplace = True)
df.dropna(inplace = True)
df['valueM€'] = df['valueM€'].str[1:-1]
df.to_sql(name = 'fifa_data', con = engine, if_exists = 'replace', index_label = 'ID')

In [ ]:
df.head()

In [ ]:
# creating 2 db tables for further normalization of data with 'declarative_base' object from SQLAlchemy’s ORM
# creating columns as attributes to each class:

In [ ]:
Base = declarative_base()  

In [ ]:
class Nationality(Base):
    __tablename__ = 'nationality_info'
    __table_args__ = {'extend_existing': True}
    
    nationality = Column(String, primary_key = True)
    flag = Column(String)
    
    def __repr__(self):
        return f'Nationality {self.name}'
    
class Club(Base):
    __tablename__ = 'club_info'
    __table_args__ = {'extend_existing': True}
    
    club = Column(String, primary_key = True)
    club_logo = Column(String)
    
    def __repr__(self):
        return f'Club {self.name}'
    
Base.metadata.create_all(engine)

In [ ]:
# previewing table names:

In [ ]:
engine.table_names()

In [ ]:
# inserting distinct data for each column from parent table to "child" tables:

In [ ]:
engine.execute("""
                INSERT INTO nationality_info
                SELECT DISTINCT nationality, flag
                FROM fifa_data;
                
                INSERT INTO club_info
                SELECT DISTINCT club, club_logo
                FROM fifa_data;""")

In [ ]:
# creating primary key <-> foreign key relations:

In [ ]:
engine.execute("""
                ALTER TABLE fifa_data
                ADD CONSTRAINT f_k_fifa_nat FOREIGN KEY (nationality) REFERENCES nationality_info(nationality);
                
                ALTER TABLE fifa_data
                ADD CONSTRAINT f_k_fifa_club FOREIGN KEY (club) REFERENCES club_info(club);""")

In [ ]:
# cleaning columns from parent table:

In [ ]:
engine.execute("""
                ALTER TABLE fifa_data
                DROP COLUMN flag;
                
                ALTER TABLE fifa_data
                DROP COLUMN club_logo; """)

In [ ]:
# previewing whole data with information_schema:

In [ ]:
engine.execute("""
                SELECT table_name, column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'fifa_data'
                OR table_name = 'club_info'
                OR table_name = 'nationality_info';""").fetchall()

In [ ]:
# performing some quering:

In [ ]:
engine.execute("""
                SELECT name, age, club, club_logo
                FROM fifa_data
                INNER JOIN club_info
                USING (club)
                WHERE age > 30
                ORDER BY age DESC
                LIMIT 10 """).fetchall()

In [ ]:
engine.execute("""
                SELECT AVG(potential) AS avg_potential, club
                FROM fifa_data
                GROUP BY club
                ORDER BY avg_potential DESC
                LIMIT 10;""").fetchall()

In [ ]:
# creating a db view, for other users:

In [ ]:
engine.execute("""
                CREATE VIEW mancity AS
                
                SELECT *
                FROM fifa_data
                INNER JOIN club_info
                USING(club)
                WHERE club = 'Manchester City';""")

In [ ]:
# creating roles, and granting access to view for other users:

In [35]:
engine.execute("""
                CREATE ROLE mancity_marketing;
                
                CREATE ROLE mancity_marketing_inter WITH PASSWORD 'change_me' VALID UNTIL '2020-12-31';
                
                GRANT mancity_marketing TO mancity_marketing_inter;
                
                GRANT UPDATE, INSERT, UPDATE, DELETE ON mancity TO mancity_marketing;""")